In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler


In [7]:
# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data.shape
data.tail(28)


,date,open,high,low,close,volume,adjusted_close,change_percent,avg_vol_20d
10755,2023-08-11,177.32,178.62,176.55,177.79,51988100,177.79,0.03,58532040.00
10756,2023-08-14,177.97,179.69,177.31,179.46,43675600,179.46,0.94,58189810.00
10757,2023-08-15,178.88,179.48,177.05,177.45,43622600,177.45,-1.12,57953250.00
10758,2023-08-16,177.13,178.54,176.50,176.57,46964900,176.57,-0.50,56276130.00
10759,2023-08-17,177.14,177.51,173.48,174.00,66062900,174.00,-1.46,56600215.00
10760,2023-08-18,172.30,175.10,171.96,174.49,61114200,174.49,0.28,56060035.00
10761,2023-08-21,175.07,176.13,173.74,175.84,46311900,175.84,0.77,56106740.00
10762,2023-08-22,177.06,177.68,176.25,177.23,42084200,177.23,0.79,56346790.00
10763,2023-08-23,178.52,181.55,178.33,181.12,52722800,181.12,2.19,56609335.00
10764,2023-08-24,180.67,181.10,176.01,176.38,54945800,176.38,-2.62,56983615.00


In [3]:
data = data.iloc[:10747]
data.tail(1)


,date,open,high,low,close,volume,adjusted_close,change_percent,avg_vol_20d
10746,2023-07-31,196.06,196.49,195.26,196.45,38824100,196.1851,0.32,49803320.0


In [4]:
data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
#data.isnull().sum()


In [5]:


#data["date"] = data["date"].astype(float)
data["open"] = data["open"].astype(float)
data["high"] = data["high"].astype(float)
data["low"] = data["low"].astype(float)
data["volume"] = data["volume"].astype(float)
data["adjusted_close"] = data["adjusted_close"].astype(float)
data["change_percent"] = data["change_percent"].astype(float)
data["avg_vol_20d"] = data["avg_vol_20d"].astype(float)

data["close"] = data["close"].astype(float)


In [6]:
# Specify the columns you want to standardize
columns_to_standardize = ["open", "high", "low", "volume", "adjusted_close", "change_percent", "avg_vol_20d"]

# Create a StandardScaler object
scaler = StandardScaler()

# Loop through the columns and standardize each one
for column in columns_to_standardize:
    data[column] = scaler.fit_transform(data[[column]])


In [7]:
import torch
import torch.nn as nn
import pandas as pd

# Sample data with a 'date' column containing dates from 1980-12-12 to 2023-07-31
data2 = pd.DataFrame({'date': pd.date_range(start='1980-12-12', end='2023-07-31')})

# Convert 'date' column to datetime
data2['date'] = pd.to_datetime(data['date'])

# Extract day, month, and year from the date column
data['day'] = data2['date'].dt.day
data['month'] = data2['date'].dt.month
data['year'] = data2['date'].dt.year

# Define the embedding dimensions
embedding_dim = 1  # You can adjust this dimension as needed

# Create embedding layers for day, month, and year
day_embedding = nn.Embedding(32, embedding_dim)  # 0-31 days
month_embedding = nn.Embedding(13, embedding_dim)  # 1-12 months
year_embedding = nn.Embedding(44, embedding_dim)  # Embedding for years from 1980 to 2023

# Convert day, month, and year to tensors with Long data type
day_tensor = torch.LongTensor(data['day'].values)
month_tensor = torch.LongTensor(data['month'].values)
year_tensor = torch.LongTensor(data['year'].values - 1980)  # Convert years to an index from 0 to 43

# Pass tensors through embedding layers to get embeddings
day_embeddings = day_embedding(day_tensor)
month_embeddings = month_embedding(month_tensor)
year_embeddings = year_embedding(year_tensor)

# Concatenate the embeddings
date_embeddings = torch.cat((day_embeddings, month_embeddings, year_embeddings), dim=1)

# Print the resulting embeddings
print(date_embeddings)


tensor([[-0.0136, -1.3303,  1.3633],
        [ 1.5315, -1.3303,  1.3633],
        [-0.8024, -1.3303,  1.3633],
        ...,
        [-1.3469, -0.0567, -0.8422],
        [ 1.6268, -0.0567, -0.8422],
        [-0.7574, -0.0567, -0.8422]], grad_fn=<CatBackward0>)


In [8]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load the x_train and y_train data
x_train = data[['open', 'high', 'low', 'volume','adjusted_close', 'change_percent', 'avg_vol_20d']].to_numpy()
y_train = data["close"].to_numpy()

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.33, random_state=45)

# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform both training and testing data
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


# Concatenate date embeddings with your testing data
x_test_feature_tensors = torch.tensor(x_test_scaled, dtype=torch.float32)
x_test_date_embeddings = date_embeddings[len(x_train_scaled):]  # Use the remaining embeddings for testing data
x_test_combined = torch.cat((x_test_feature_tensors, x_test_date_embeddings), dim=1)

# Convert the combined testing data to PyTorch tensors
x_test_tensor = torch.tensor(x_test_combined, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)






# Inside your LSTM model class
class LSTMModel(nn.Module):
    def __init__(self, input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length):
        super(LSTMModel, self).__init__()

        self.input_size = input_size + date_embedding_dim  # Updated input size to include date embeddings
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.sequence_length = sequence_length

        # Define the LSTM layer as a class attribute
        self.lstm = nn.LSTM(self.input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        batch_size = x.size(0)

        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)

        # Ensure input has the shape [batch_size, sequence_length, input_size]
        x = x.view(batch_size, 1, self.input_size)

        out, (hidden, cell) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out




# Set random seeds for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# Define hyperparameters
input_size = 7  # Adjust based on the number of input features (excluding change_percent)
date_embedding_dim = 3  # Adjust based on the dimension of your date embeddings
output_size = 1
hidden_dim = 16
n_layers = 4
sequence_length = 75  # Keep this as 1 for your input data
batch_size = 64

# Concatenate date embeddings with your feature vectors
x_train_feature_tensors = torch.tensor(x_train_scaled, dtype=torch.float32)
x_train_date_embeddings = date_embeddings[:len(x_train_scaled)]  # Use the same length as your training data
x_train_combined = torch.cat((x_train_feature_tensors, x_train_date_embeddings), dim=1)




# Convert the combined data to PyTorch tensors
#x_train_tensor = torch.tensor(x_train_combined, dtype=torch.float32)
#y_train_tensor = torch.tensor(y_train, dtype=torch.float32)



# Copy x_train_combined to create x_train_tensor
x_train_tensor = x_train_combined.clone().detach()
x_train_tensor = x_train_tensor.to(torch.float32)

# Copy x_test_combined to create x_test_tensor
x_test_tensor = x_test_combined.clone().detach()
x_test_tensor = x_test_tensor.to(torch.float32)


# Create y_train_tensor directly from y_train
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)





# Create a DataLoader for batch training
train_data = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)




# Create an instance of the LSTM model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length).to(device)

# Define loss function and optimizer
loss_function = nn.MSELoss()
learning_rate = 0.00015
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(f'Hyperparameters: Learning Rate={learning_rate}, Sequence Length={sequence_length}, Batch Size={batch_size}, Input Size={input_size}, Date Embedding Dim={date_embedding_dim}, Hidden Dim={hidden_dim},'
              f'Layers={n_layers}')


# Training loop
num_epochs = 6000

for epoch in range(num_epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_x)

        # Calculate loss
        loss = loss_function(outputs, batch_y.view(-1, 1))  # Ensure batch_y has the right shape

        # Backpropagation
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        # Extract features on the testing set
        model.eval()
        with torch.no_grad():
            # Extract features from the hidden states
            hidden_states, _ = model.lstm(x_test_tensor.view(x_test_tensor.size(0), 1, -1))
            val_outputs = model(x_test_tensor)
            val_loss = loss_function(val_outputs, y_test_tensor.view(-1, 1))  # Ensure y_test_tensor has the right shape
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')
        # You can now use 'hidden_states' as the feature representations of your sequences
        # The shape of 'hidden_states' will be (batch_size, sequence_length, hidden_dim)

# Print hidden_states after training
print(hidden_states)










C:\Users\pitr7\AppData\Local\Temp\ipykernel_7272\3653438643.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test_tensor = torch.tensor(x_test_combined, dtype=torch.float32)


Hyperparameters: Learning Rate=0.00015, Sequence Length=75, Batch Size=64, Input Size=7, Date Embedding Dim=3, Hidden Dim=16,Layers=4


1. Feature Extraction:

In [ ]:
model.eval()
with torch.no_grad():
    hidden_states, _ = model.lstm(x_test_tensor.view(x_test_tensor.size(0), 1, -1))
    val_outputs = model(x_test_tensor)
    y_test_predictions = val_outputs


2. generate date embeddings for the future date, 11th August

In [ ]:
import torch

# Define the target date: 12th August
target_date = "11th August"

# Define the embedding dimensions (same as in your previous code)
embedding_dim = 1

# Define the maximum values for day, month, and year based on your previous code
max_day = 31  # Maximum day
max_month = 12  # Maximum month
max_year = 43  # Maximum year (from 1980 to 2023)

# Create one-hot encodings for day, month, and year
day_encoding = torch.zeros(max_day)
month_encoding = torch.zeros(max_month)
year_encoding = torch.zeros(max_year + 1)  # +1 to account for the inclusive range

# Map the target date to one-hot encoding
# Extract the day and month from the target date
day_index = int(target_date.split(" ")[0].replace("th", "")) - 1  # Extract the day from the target date
month_index = 8  # August is the 9th month (0-based index)

# Set the corresponding elements to 1
day_encoding[day_index] = 1
month_encoding[month_index] = 1
year_encoding[43] = 1  # 43 corresponds to the year 2023 in your previous code

# Concatenate the day, month, and year encodings to get the date embedding
date_embedding_11_august = torch.cat((day_encoding, month_encoding, year_encoding), dim=0)



3. Combine LSTM Features with Date Embeddings

In [ ]:

import torch

# Assuming 'hidden_states' has shape [3547, 1, 32]
# 'date_embedding_11_august' has shape [76]

# Broadcast 'date_embedding_11_august' to match the shape of 'hidden_states'
# This will repeat 'date_embedding_11_august' along the second dimension
date_embedding_11_august_broadcasted = date_embedding_11_august.reshape(1, 1, -1).expand(3547, 1, -1)

# Combine 'hidden_states' and 'date_embedding_11_august_broadcasted'
combined_states = torch.cat((hidden_states, date_embedding_11_august_broadcasted), dim=2)

# The resulting 'combined_states' will have a shape of [3547, 1, 32 + 76]


In [ ]:

# Check the shape of 'combined_states_flat'
##print("Shape of combined_states_flat:", combined_states_flat.shape)

# Check the shape of 'y_train'
##print("Shape of y_train:", y_train.shape)


4. Train a Random Forest Model:

In [ ]:
import torch
import numpy as np
from sklearn.ensemble import RandomForestRegressor

# Assuming 'y_test_predictions' contains the predicted 'close' values
# You can extract them as a numpy array using .numpy() method
close_values = y_test_predictions.numpy()

# Reshape the 'combined_states' to remove the extra dimension
combined_states_2d = combined_states.reshape(-1, combined_states.shape[-1])

# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)  # You can adjust hyperparameters as needed

# Fit the model on your data
rf_model.fit(combined_states_2d, close_values)

# Now the Random Forest model is trained and ready for predictions


C:\Users\pitr7\AppData\Local\Temp\ipykernel_3500\696125167.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(combined_states_2d, close_values)


RandomForestRegressor(random_state=42)

5 Evaluate the model

In [ ]:
import torch
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Assuming 'y_test_predictions' contains the predicted 'close' values
# You can extract them as a numpy array using .numpy() method
close_values = y_test_predictions.numpy()

# Reshape the 'combined_states' to remove the extra dimension
combined_states_2d = combined_states.reshape(-1, combined_states.shape[-1])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_states_2d, close_values, test_size=0.2, random_state=42)

# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)  # You can adjust hyperparameters as needed

# Fit the model on your training data
rf_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf_model.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Print or use the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R-squared (R^2): {r2}')




C:\Users\pitr7\AppData\Local\Temp\ipykernel_3500\247992320.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(X_train, y_train)


Mean Absolute Error (MAE): 0.1974559440747116
Mean Squared Error (MSE): 0.13670674518134904
Root Mean Squared Error (RMSE): 0.36973875260966227
R-squared (R^2): 0.9999927374686682


6. Make Predictions for 11th August:

In [ ]:
import torch
import numpy as np
from sklearn.ensemble import RandomForestRegressor

# Assuming 'hidden_states' contains the LSTM features
# 'y_test_predictions' contains the predicted 'close' values

# Define the target date: 11th August
target_date = "11th August"

# Define the embedding dimensions (same as in your previous code)
embedding_dim = 1

# Define the maximum values for day, month, and year based on your previous code
max_day = 31  # Maximum day
max_month = 12  # Maximum month
max_year = 43  # Maximum year (from 1980 to 2023)

# Create one-hot encodings for day, month, and year
day_encoding = torch.zeros(max_day)
month_encoding = torch.zeros(max_month)
year_encoding = torch.zeros(max_year + 1)  # +1 to account for the inclusive range

# Map the target date to one-hot encoding
# Extract the day and month from the target date
day_index = int(target_date.split(" ")[0].replace("th", "")) - 1  # Extract the day from the target date
month_index = 8  # August is the 9th month (0-based index)

# Set the corresponding elements to 1
day_encoding[day_index] = 1
month_encoding[month_index] = 1
year_encoding[43] = 1  # 43 corresponds to the year 2023 in your previous code

# Concatenate the day, month, and year encodings to get the date embedding
date_embedding_11_august = torch.cat((day_encoding, month_encoding, year_encoding), dim=0)

# Broadcast 'date_embedding_11_august' to match the shape of 'hidden_states'
date_embedding_11_august_broadcasted = date_embedding_11_august.reshape(1, 1, -1).expand(hidden_states.shape[0], 1, -1)

# Combine 'hidden_states' and 'date_embedding_11_august_broadcasted'
combined_states = torch.cat((hidden_states, date_embedding_11_august_broadcasted), dim=2)

# Extract 'y_test_predictions' as a numpy array
close_values = y_test_predictions.numpy()

# Reshape the 'combined_states' to remove the extra dimension
combined_states_2d = combined_states.reshape(-1, combined_states.shape[-1])

# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model on your data
rf_model.fit(combined_states_2d, close_values)

# Now, the Random Forest model is trained and ready for predictions

# Make a prediction for 11th August 2023
predicted_close_11_august = rf_model.predict(combined_states_2d)

# 'predicted_close_11_august' contains the predicted "close" value for 11th August 2023
# Print the predicted "close" value for 11th August 2023
print("Predicted Close Value for 11th August 2023:", predicted_close_11_august)


C:\Users\pitr7\AppData\Local\Temp\ipykernel_3500\691711526.py:53: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(combined_states_2d, close_values)


Predicted Close Value for 11th August 2023: [ 41.53316753  43.02049961  28.32345577 ... 112.46756454  27.92956419
 174.02247238]


In [ ]:
import torch
import numpy as np
from sklearn.ensemble import RandomForestRegressor

# Assuming 'hidden_states' contains the LSTM features
# 'y_test_predictions' contains the predicted 'close' values

# Define the target date: 12th August
target_date = "12th August"

# Define the embedding dimensions (same as in your previous code)
embedding_dim = 1

# Define the maximum values for day, month, and year based on your previous code
max_day = 31  # Maximum day
max_month = 12  # Maximum month
max_year = 43  # Maximum year (from 1980 to 2023)

# Create one-hot encodings for day, month, and year
day_encoding = torch.zeros(max_day)
month_encoding = torch.zeros(max_month)
year_encoding = torch.zeros(max_year + 1)  # +1 to account for the inclusive range

# Map the target date to one-hot encoding
# Extract the day and month from the target date
day_index = int(target_date.split(" ")[0].replace("th", "")) - 1  # Extract the day from the target date
month_index = 8  # August is the 9th month (0-based index)

# Set the corresponding elements to 1
day_encoding[day_index] = 1
month_encoding[month_index] = 1
year_encoding[43] = 1  # 43 corresponds to the year 2023 in your previous code

# Concatenate the day, month, and year encodings to get the date embedding
date_embedding_12_august = torch.cat((day_encoding, month_encoding, year_encoding), dim=0)

# Broadcast 'date_embedding_12_august' to match the shape of 'hidden_states'
date_embedding_12_august_broadcasted = date_embedding_12_august.reshape(1, 1, -1).expand(hidden_states.shape[0], 1, -1)

# Combine 'hidden_states' and 'date_embedding_12_august_broadcasted'
combined_states = torch.cat((hidden_states, date_embedding_12_august_broadcasted), dim=2)

# Extract 'y_test_predictions' as a numpy array
close_values = y_test_predictions.numpy()

# Reshape the 'combined_states' to remove the extra dimension
combined_states_2d = combined_states.reshape(-1, combined_states.shape[-1])

# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model on your data
rf_model.fit(combined_states_2d, close_values)

# Now, the Random Forest model is trained and ready for predictions

# Make a prediction for 12th August 2023
predicted_close_12_august = rf_model.predict(combined_states_2d)

# 'predicted_close_12_august' contains the predicted "close" value for 12th August 2023
# Print the predicted "close" value for 12th August 2023
print("Predicted Close Value for 12th August 2023:", predicted_close_12_august)


C:\Users\pitr7\AppData\Local\Temp\ipykernel_3500\2648064599.py:53: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(combined_states_2d, close_values)


Predicted Close Value for 12th August 2023: [ 41.53316753  43.02049961  28.32345577 ... 112.46756454  27.92956419
 174.02247238]


In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor


# Reshape the hidden states
reshaped_features = hidden_states.view(x_test_tensor.size(0), -1).numpy()

# List of models to iterate over
models = [
    RandomForestRegressor(n_estimators=100, random_state=42),


]

for model in models:
    model_name = type(model).__name__
    # Initialize and train the current model
    model.fit(reshaped_features, y_test)

    # Make predictions
    model_predictions = model.predict(reshaped_features)

    # Calculate RMSE
    rmse = mean_squared_error(y_test, model_predictions, squared=False)

    # Print RMSE for the current model
    print(f"Model: {model_name}, RMSE: {rmse}")


ValueError: Found input variables with inconsistent numbers of samples: [3547, 710]

: 

: 

: 

: 

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, HuberRegressor, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import TweedieRegressor
from sklearn.metrics import mean_squared_error

# Reshape the hidden states
reshaped_features = hidden_states.view(x_test_tensor.size(0), -1).numpy()

# List of models to iterate over
models = [
    RandomForestRegressor(n_estimators=100, random_state=42),
    xgb.XGBRegressor(),
    LinearRegression(),
    Ridge(alpha=1.0),
    Lasso(alpha=1.0),
    xgb.XGBRegressor(n_estimators=100, random_state=42),
    KNeighborsRegressor(n_neighbors=5),
    DecisionTreeRegressor(),

    HuberRegressor(),
    SVR(),
    GaussianProcessRegressor(),
    SGDRegressor(),
    TweedieRegressor(power=1, alpha=0.5),
    AdaBoostRegressor(n_estimators=100, random_state=42),
    GradientBoostingRegressor(n_estimators=100, random_state=42),

    OrthogonalMatchingPursuit(),
    IsotonicRegression()
]

for model in models:
    model_name = type(model).__name__
    # Initialize and train the current model
    model.fit(reshaped_features, y_test)

    # Make predictions
    model_predictions = model.predict(reshaped_features)

    # Calculate RMSE
    rmse = mean_squared_error(y_test, model_predictions, squared=False)

    # Print RMSE for the current model
    print(f"Model: {model_name}, RMSE: {rmse}")


: 

: 

In [ ]:

use the model to make predictions for future time steps.
You input historical sequences of features, and the model generates predictions for future target values.

Use the features extracted from the LSTM as input to a DecissiontreeRegressor model.
Train these models to predict the target variable, close, using the LSTM-derived features.
close value, on the future 12 agost 2023
We dont have the feature values in the future



# Specify the columns you want to standardize
columns_to_standardize = ["open", "high", "low", "volume", "adjusted_close", "change_percent", "avg_vol_20d"]

# Create a StandardScaler object
scaler = StandardScaler()

# Loop through the columns and standardize each one
for column in columns_to_standardize:
    data[column] = scaler.fit_transform(data[[column]])




import torch
import torch.nn as nn
import pandas as pd

# Sample data with a 'date' column containing dates from 1980-12-12 to 2023-08-31
data2 = pd.DataFrame({'date': pd.date_range(start='1980-12-12', end='2023-08-31')})

# Convert 'date' column to datetime
data2['date'] = pd.to_datetime(data['date'])

# Extract day, month, and year from the date column
data['day'] = data2['date'].dt.day
data['month'] = data2['date'].dt.month
data['year'] = data2['date'].dt.year

# Define the embedding dimensions
embedding_dim = 1  # You can adjust this dimension as needed

# Create embedding layers for day, month, and year
day_embedding = nn.Embedding(32, embedding_dim)  # 0-31 days
month_embedding = nn.Embedding(13, embedding_dim)  # 1-12 months
year_embedding = nn.Embedding(44, embedding_dim)  # Embedding for years from 1980 to 2023

# Convert day, month, and year to tensors with Long data type
day_tensor = torch.LongTensor(data['day'].values)
month_tensor = torch.LongTensor(data['month'].values)
year_tensor = torch.LongTensor(data['year'].values - 1980)  # Convert years to an index from 0 to 43

# Pass tensors through embedding layers to get embeddings
day_embeddings = day_embedding(day_tensor)
month_embeddings = month_embedding(month_tensor)
year_embeddings = year_embedding(year_tensor)

# Concatenate the embeddings
date_embeddings = torch.cat((day_embeddings, month_embeddings, year_embeddings), dim=1)

# Print the resulting embeddings
print(date_embeddings)


import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load the x_train and y_train data
x_train = data[['open', 'high', 'low', 'volume','adjusted_close', 'change_percent', 'avg_vol_20d']].to_numpy()
y_train = data["close"].to_numpy()

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.33, random_state=45)

# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform both training and testing data
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


# Concatenate date embeddings with your testing data
x_test_feature_tensors = torch.tensor(x_test_scaled, dtype=torch.float32)
x_test_date_embeddings = date_embeddings[len(x_train_scaled):]  # Use the remaining embeddings for testing data
x_test_combined = torch.cat((x_test_feature_tensors, x_test_date_embeddings), dim=1)

# Convert the combined testing data to PyTorch tensors
x_test_tensor = torch.tensor(x_test_combined, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)






# Inside your LSTM model class
class LSTMModel(nn.Module):
    def __init__(self, input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length):
        super(LSTMModel, self).__init__()

        self.input_size = input_size + date_embedding_dim  # Updated input size to include date embeddings
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.sequence_length = sequence_length

        # Define the LSTM layer as a class attribute
        self.lstm = nn.LSTM(self.input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        batch_size = x.size(0)

        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)

        # Ensure input has the shape [batch_size, sequence_length, input_size]
        x = x.view(batch_size, 1, self.input_size)

        out, (hidden, cell) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out




# Set random seeds for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# Define hyperparameters
input_size = 7  # Adjust based on the number of input features (excluding change_percent)
date_embedding_dim = 3  # Adjust based on the dimension of your date embeddings
output_size = 1
hidden_dim = 32
n_layers = 4
sequence_length = 75  # Keep this as 1 for your input data
batch_size = 64

# Concatenate date embeddings with your feature vectors
x_train_feature_tensors = torch.tensor(x_train_scaled, dtype=torch.float32)
x_train_date_embeddings = date_embeddings[:len(x_train_scaled)]  # Use the same length as your training data
x_train_combined = torch.cat((x_train_feature_tensors, x_train_date_embeddings), dim=1)




# Convert the combined data to PyTorch tensors
#x_train_tensor = torch.tensor(x_train_combined, dtype=torch.float32)
#y_train_tensor = torch.tensor(y_train, dtype=torch.float32)



# Copy x_train_combined to create x_train_tensor
x_train_tensor = x_train_combined.clone().detach()
x_train_tensor = x_train_tensor.to(torch.float32)

# Copy x_test_combined to create x_test_tensor
x_test_tensor = x_test_combined.clone().detach()
x_test_tensor = x_test_tensor.to(torch.float32)


# Create y_train_tensor directly from y_train
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)





# Create a DataLoader for batch training
train_data = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)




# Create an instance of the LSTM model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_size, date_embedding_dim, hidden_dim, n_layers, output_size, sequence_length).to(device)

# Define loss function and optimizer
loss_function = nn.MSELoss()
learning_rate = 0.00015
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(f'Hyperparameters: Learning Rate={learning_rate}, Sequence Length={sequence_length}, Batch Size={batch_size}, Input Size={input_size}, Date Embedding Dim={date_embedding_dim}, Hidden Dim={hidden_dim},'
              f'Layers={n_layers}')


# Training loop
num_epochs = 3000

for epoch in range(num_epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_x)

        # Calculate loss
        loss = loss_function(outputs, batch_y.view(-1, 1))  # Ensure batch_y has the right shape

        # Backpropagation
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        # Extract features on the testing set
        model.eval()
        with torch.no_grad():
            # Extract features from the hidden states
            hidden_states, _ = model.lstm(x_test_tensor.view(x_test_tensor.size(0), 1, -1))
            val_outputs = model(x_test_tensor)
            val_loss = loss_function(val_outputs, y_test_tensor.view(-1, 1))  # Ensure y_test_tensor has the right shape
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')
        # You can now use 'hidden_states' as the feature representations of your sequences
        # The shape of 'hidden_states' will be (batch_size, sequence_length, hidden_dim)

# Print hidden_states after training
print(hidden_states)













: 

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load the x_train and y_train data (as you have already done)
x_train = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d'].to_numpy()
y_train = data["close"].to_numpy()

# Split the data into training and testing sets (as you have already done)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.33, random_state=45)

# Initialize the scaler (as you have already done)
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Create a PyTorch tensor for the testing data (as you have already done)
x_test_feature_tensors = torch.tensor(x_test_scaled, dtype=torch.float32)

# Assume you have the LSTM-derived features in 'hidden_states' as you did in your code

# Create a DecisionTreeRegressor model
decision_tree_model = DecisionTreeRegressor(random_state=42)

# Fit the model with the LSTM-derived features
decision_tree_model.fit(hidden_states.numpy(), y_train)  # Assuming 'y_train' contains your training labels

# Use the LSTM-derived features from the last time step for August 12, 2023
# Replace these placeholders with the actual data from your LSTM model
features_august_12 = ...  # Replace with the LSTM-derived features for August 12, 2023

# Now, you can use the LSTM-derived features to make predictions with the DecisionTreeRegressor
predicted_close = decision_tree_model.predict([features_august_12])

# 'predicted_close' will contain the predicted close value for August 12, 2023
print(f"Predicted close on August 12, 2023: {predicted_close[0]}")


: 